In [1]:
import sys
import pathlib as pl
import pickle
import warnings

import pandas as pd
import numpy as np
import gtfs_kit as gk
import folium as fl

sys.path.append('../')

import make_gtfs as mg


DATA_DIR = pl.Path('../data')

%load_ext autoreload
%autoreload 2

warnings.filterwarnings(action='ignore')

/Users/jtltransitlab/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
path = DATA_DIR / 'khartoum'
pfeed = mg.read_protofeed(path)
pfeed

ProtoFeed(meta=  agency_name                                    agency_url  agency_timezone  \
0  KhartouMap  https://www.linkedin.com/company/khartoumap/  Africa/Khartoum   

  start_date  end_date  
0   20230101  20280101  , service_windows=  service_window_id start_time  end_time  monday  tuesday  wednesday  \
0      weekday_peak   06:30:00  08:59:59       1        1          1   
1   weekday_offpeak   09:00:00  23:59:59       1        1          1   
2           weekend   07:00:00  23:59:59       0        0          0   

   thursday  friday  saturday  sunday  
0         1       0         0       1  
1         1       0         0       1  
2         0       1         1       0  , shapes=    shape_id                                           geometry
0      shp_4  LINESTRING (32.52187 15.59309, 32.52191 15.593...
1      shp_5  LINESTRING (32.52187 15.59309, 32.52187 15.593...
2      shp_6  LINESTRING (32.52187 15.59309, 32.52187 15.593...
3      shp_8  LINESTRING (32.52187 15.59309,

In [3]:
%%time

feed = mg.build_feed(pfeed)

CPU times: user 2h 14min 35s, sys: 7min 35s, total: 2h 22min 10s
Wall time: 2h 23min 25s


In [4]:
feed.agency.to_csv('agency.txt', index= False)
feed.calendar.to_csv('calendar.txt', index= False)
feed.routes.to_csv('routes.txt', index= False)
feed.shapes.to_csv('shapes.txt', index= False)
feed.trips.to_csv('trips.txt', index= False)
feed.stops.to_csv('stops.txt', index= False)
feed.stop_times.to_csv('stop_times.txt', index= False)

In [5]:
#with open('gtfs_feed_v2-3', 'wb') as handle:
#    pickle.dump(feed, handle, protocol=pickle.HIGHEST_PROTOCOL)    

with open('gtfs_feed_v2-3', 'rb') as handle:
    feed= pickle.load(handle)

In [6]:
x= pd.read_csv('stop_times.txt')
x

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,shape_dist_traveled
0,t-rKar_24-weekend-07:00:00-0-0,shp_102-0-0,0,07:00:00,07:00:00,1.723866e-10
1,t-rKar_24-weekend-07:00:00-0-0,shp_27-0-1,1,07:00:00,07:00:00,3.664123e+00
2,t-rKar_24-weekend-07:00:00-0-0,shp_28-0-1,2,07:00:00,07:00:00,3.937130e+00
3,t-rKar_24-weekend-07:00:00-0-0,shp_119-0-0,3,07:00:00,07:00:00,4.998854e+00
4,t-rKar_24-weekend-07:00:00-0-0,shp_117-0-1,4,07:17:00,07:17:00,8.502246e+03
...,...,...,...,...,...,...
205897,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_330-0-1,3,24:04:30,24:04:30,1.725028e+04
205898,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_113-0-0,4,24:13:09,24:13:09,2.157603e+04
205899,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_127-0-1,5,24:17:26,24:17:26,2.372097e+04
205900,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_111-0-0,6,24:20:21,24:20:21,2.517639e+04


In [7]:
x.drop_duplicates(inplace= True)
x

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,shape_dist_traveled
0,t-rKar_24-weekend-07:00:00-0-0,shp_102-0-0,0,07:00:00,07:00:00,1.723866e-10
1,t-rKar_24-weekend-07:00:00-0-0,shp_27-0-1,1,07:00:00,07:00:00,3.664123e+00
2,t-rKar_24-weekend-07:00:00-0-0,shp_28-0-1,2,07:00:00,07:00:00,3.937130e+00
3,t-rKar_24-weekend-07:00:00-0-0,shp_119-0-0,3,07:00:00,07:00:00,4.998854e+00
4,t-rKar_24-weekend-07:00:00-0-0,shp_117-0-1,4,07:17:00,07:17:00,8.502246e+03
...,...,...,...,...,...,...
205889,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_330-0-1,3,24:04:30,24:04:30,1.725028e+04
205890,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_113-0-0,4,24:13:09,24:13:09,2.157603e+04
205891,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_127-0-1,5,24:17:26,24:17:26,2.372097e+04
205892,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_111-0-0,6,24:20:21,24:20:21,2.517639e+04


In [8]:
x= x[x.shape_dist_traveled > 0.01]
x

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,shape_dist_traveled
1,t-rKar_24-weekend-07:00:00-0-0,shp_27-0-1,1,07:00:00,07:00:00,3.664123
2,t-rKar_24-weekend-07:00:00-0-0,shp_28-0-1,2,07:00:00,07:00:00,3.937130
3,t-rKar_24-weekend-07:00:00-0-0,shp_119-0-0,3,07:00:00,07:00:00,4.998854
4,t-rKar_24-weekend-07:00:00-0-0,shp_117-0-1,4,07:17:00,07:17:00,8502.245663
11,t-rKar_24-weekend-07:00:00-0-1,shp_27-0-1,1,07:20:00,07:20:00,3.664123
...,...,...,...,...,...,...
205889,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_330-0-1,3,24:04:30,24:04:30,17250.281204
205890,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_113-0-0,4,24:13:09,24:13:09,21576.026654
205891,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_127-0-1,5,24:17:26,24:17:26,23720.968657
205892,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_111-0-0,6,24:20:21,24:20:21,25176.387267


In [9]:
x.drop_duplicates(subset=['trip_id', 'shape_dist_traveled'], keep= 'first', inplace= True)
x

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,shape_dist_traveled
1,t-rKar_24-weekend-07:00:00-0-0,shp_27-0-1,1,07:00:00,07:00:00,3.664123
2,t-rKar_24-weekend-07:00:00-0-0,shp_28-0-1,2,07:00:00,07:00:00,3.937130
3,t-rKar_24-weekend-07:00:00-0-0,shp_119-0-0,3,07:00:00,07:00:00,4.998854
4,t-rKar_24-weekend-07:00:00-0-0,shp_117-0-1,4,07:17:00,07:17:00,8502.245663
11,t-rKar_24-weekend-07:00:00-0-1,shp_27-0-1,1,07:20:00,07:20:00,3.664123
...,...,...,...,...,...,...
205889,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_330-0-1,3,24:04:30,24:04:30,17250.281204
205890,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_113-0-0,4,24:13:09,24:13:09,21576.026654
205891,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_127-0-1,5,24:17:26,24:17:26,23720.968657
205892,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_111-0-0,6,24:20:21,24:20:21,25176.387267


In [10]:
x.drop_duplicates(subset=['trip_id', 'arrival_time'], keep= 'first', inplace= True)
x

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,shape_dist_traveled
0,t-rKar_24-weekend-07:00:00-0-0,shp_27-0-1,1,07:00:00,07:00:00,3.664123
1,t-rKar_24-weekend-07:00:00-0-0,shp_117-0-1,4,07:17:00,07:17:00,8502.245663
2,t-rKar_24-weekend-07:00:00-0-1,shp_27-0-1,1,07:20:00,07:20:00,3.664123
3,t-rKar_24-weekend-07:00:00-0-1,shp_117-0-1,4,07:37:00,07:37:00,8502.245663
4,t-rKar_24-weekend-07:00:00-0-2,shp_27-0-1,1,07:40:00,07:40:00,3.664123
...,...,...,...,...,...,...
70966,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_330-0-1,3,24:04:30,24:04:30,17250.281204
70967,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_113-0-0,4,24:13:09,24:13:09,21576.026654
70968,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_127-0-1,5,24:17:26,24:17:26,23720.968657
70969,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_111-0-0,6,24:20:21,24:20:21,25176.387267


In [29]:
subset= x.groupby('trip_id').count()
x = x[x.trip_id.isin(subset[subset.stop_id > 1].index.values)]
x.reset_index(drop= True, inplace= True)
x

,trip_id,stop_id,stop_sequence,arrival_time,departure_time,shape_dist_traveled
0,t-rKar_24-weekend-07:00:00-0-0,shp_27-0-1,1,07:00:00,07:00:00,3.664123
1,t-rKar_24-weekend-07:00:00-0-0,shp_117-0-1,4,07:17:00,07:17:00,8502.245663
2,t-rKar_24-weekend-07:00:00-0-1,shp_27-0-1,1,07:20:00,07:20:00,3.664123
3,t-rKar_24-weekend-07:00:00-0-1,shp_117-0-1,4,07:37:00,07:37:00,8502.245663
4,t-rKar_24-weekend-07:00:00-0-2,shp_27-0-1,1,07:40:00,07:40:00,3.664123
...,...,...,...,...,...,...
55837,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_330-0-1,3,24:04:30,24:04:30,17250.281204
55838,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_113-0-0,4,24:13:09,24:13:09,21576.026654
55839,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_127-0-1,5,24:17:26,24:17:26,23720.968657
55840,t-rKar_03-weekday_offpeak-09:00:00-1-58,shp_111-0-0,6,24:20:21,24:20:21,25176.387267


In [30]:
x.to_csv('stop_times.txt', index= False)

In [31]:
y= pd.read_csv('trips.txt')
y

,route_id,trip_id,direction_id,shape_id,service_id
0,rkrt_01,t-rkrt_01-weekend-07:00:00-0-0,0,shp_4-0,srv0000110
1,rkrt_01,t-rkrt_01-weekend-07:00:00-0-1,0,shp_4-0,srv0000110
2,rkrt_01,t-rkrt_01-weekend-07:00:00-0-2,0,shp_4-0,srv0000110
3,rkrt_01,t-rkrt_01-weekend-07:00:00-0-3,0,shp_4-0,srv0000110
4,rkrt_01,t-rkrt_01-weekend-07:00:00-0-4,0,shp_4-0,srv0000110
...,...,...,...,...,...
31729,rJBL_85,t-rJBL_85-weekday_offpeak-09:00:00-1-54,1,shp_417-1,srv1111001
31730,rJBL_85,t-rJBL_85-weekday_offpeak-09:00:00-1-55,1,shp_417-1,srv1111001
31731,rJBL_85,t-rJBL_85-weekday_offpeak-09:00:00-1-56,1,shp_417-1,srv1111001
31732,rJBL_85,t-rJBL_85-weekday_offpeak-09:00:00-1-57,1,shp_417-1,srv1111001


In [32]:
y= y[y.trip_id.isin(x.trip_id.values)]
y.reset_index(drop= True, inplace= True)
y

,route_id,trip_id,direction_id,shape_id,service_id
0,rkrt_01,t-rkrt_01-weekend-07:00:00-0-0,0,shp_4-0,srv0000110
1,rkrt_01,t-rkrt_01-weekend-07:00:00-0-1,0,shp_4-0,srv0000110
2,rkrt_01,t-rkrt_01-weekend-07:00:00-0-2,0,shp_4-0,srv0000110
3,rkrt_01,t-rkrt_01-weekend-07:00:00-0-3,0,shp_4-0,srv0000110
4,rkrt_01,t-rkrt_01-weekend-07:00:00-0-4,0,shp_4-0,srv0000110
...,...,...,...,...,...
16600,rJBL_85,t-rJBL_85-weekday_offpeak-09:00:00-1-54,1,shp_417-1,srv1111001
16601,rJBL_85,t-rJBL_85-weekday_offpeak-09:00:00-1-55,1,shp_417-1,srv1111001
16602,rJBL_85,t-rJBL_85-weekday_offpeak-09:00:00-1-56,1,shp_417-1,srv1111001
16603,rJBL_85,t-rJBL_85-weekday_offpeak-09:00:00-1-57,1,shp_417-1,srv1111001


In [33]:
y.to_csv('trips.txt', index= False)

In [34]:
z= pd.read_csv('shapes.txt')
z

,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat
0,shp_4-0,0,32.543658,15.520905
1,shp_4-0,1,32.543496,15.521883
2,shp_4-0,2,32.543434,15.522240
3,shp_4-0,3,32.543360,15.522666
4,shp_4-0,4,32.543210,15.523549
...,...,...,...,...
63236,shp_610-0,255,32.418999,15.645684
63237,shp_610-0,256,32.419542,15.645656
63238,shp_610-0,257,32.419505,15.645423
63239,shp_610-0,258,32.419879,15.645416


In [35]:
z.drop_duplicates(subset= ['shape_id', 'shape_pt_lon', 'shape_pt_lat'], keep= 'first', inplace= True)
z

,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat
0,shp_4-0,0,32.543658,15.520905
1,shp_4-0,1,32.543496,15.521883
2,shp_4-0,2,32.543434,15.522240
3,shp_4-0,3,32.543360,15.522666
4,shp_4-0,4,32.543210,15.523549
...,...,...,...,...
63236,shp_610-0,255,32.418999,15.645684
63237,shp_610-0,256,32.419542,15.645656
63238,shp_610-0,257,32.419505,15.645423
63239,shp_610-0,258,32.419879,15.645416


In [36]:
z= z[z.shape_id.isin(y.shape_id.values)]
z.reset_index(drop= True, inplace= True)
z

,shape_id,shape_pt_sequence,shape_pt_lon,shape_pt_lat
0,shp_4-0,0,32.543658,15.520905
1,shp_4-0,1,32.543496,15.521883
2,shp_4-0,2,32.543434,15.522240
3,shp_4-0,3,32.543360,15.522666
4,shp_4-0,4,32.543210,15.523549
...,...,...,...,...
37420,shp_600-1,268,32.287586,15.589656
37421,shp_600-1,269,32.286836,15.589654
37422,shp_600-1,270,32.286099,15.589650
37423,shp_600-1,271,32.286009,15.589653


In [37]:
z.to_csv('shapes.txt', index= False)

In [19]:
f = (
    feed.routes[['route_id', 'route_short_name']]
    .merge(pfeed.frequencies)
    .merge(pfeed.service_windows)
    .sort_values('route_id')
)
f

,route_id,route_short_name,route_long_name,route_type,service_window_id,direction,frequency,shape_id,speed,start_time,end_time,monday,tuesday,wednesday,thursday,friday,saturday,sunday
256,rJBL_01,JBL_01,JBL_01,3,weekend,0,3,shp_357,30.0,07:00:00,23:59:59,0,0,0,0,1,1,0
571,rJBL_01,JBL_01,JBL_01,3,weekday_peak,1,6,shp_357,30.0,06:30:00,08:59:59,1,1,1,1,0,0,1
257,rJBL_01,JBL_01,JBL_01,3,weekend,1,3,shp_357,30.0,07:00:00,23:59:59,0,0,0,0,1,1,0
884,rJBL_01,JBL_01,JBL_01,3,weekday_offpeak,0,4,shp_357,30.0,09:00:00,23:59:59,1,1,1,1,0,0,1
885,rJBL_01,JBL_01,JBL_01,3,weekday_offpeak,1,4,shp_357,30.0,09:00:00,23:59:59,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,rkrt_34,krt_34,markazi_thawra,3,weekday_offpeak,0,4,shp_370,30.0,09:00:00,23:59:59,1,1,1,1,0,0,1
42,rkrt_34,krt_34,markazi_thawra,3,weekend,0,3,shp_370,30.0,07:00:00,23:59:59,0,0,0,0,1,1,0
671,rkrt_34,krt_34,markazi_thawra,3,weekday_offpeak,1,4,shp_370,30.0,09:00:00,23:59:59,1,1,1,1,0,0,1
43,rkrt_34,krt_34,markazi_thawra,3,weekend,1,3,shp_370,30.0,07:00:00,23:59:59,0,0,0,0,1,1,0


In [20]:
# Map some trips
tids = list(feed.trips.sample(200).trip_id.values)
feed.map_trips(tids, include_arrows=True, include_stops=True) 